# Projeto #1 - Mineração de agrupamentos

<p style='text-align: justify;'><font size=3.>Análise de acidentes nas rodovias federais brasileira entre 2007 e 2021.</font></p>

### Bibliotecas básicas e outros imports

In [ ]:
import warnings
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

%run modules/text.py

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 200)
np.set_printoptions(suppress=True, precision=3)

### Carregamento dos dados

In [ ]:
df = pd.read_csv('dataset/cluster_acidentes_rodovias_brasileiras_2007_a_2021.csv', low_memory=False, encoding='utf-8')
print(f'Linhas: {df.shape[0]} | Coluna:  {df.shape[1]}')
df.head(1)

### Categorização da variável idade

> <p style='text-align: justify;'><font size=2.85>Categorização realizada de acordo com o estatuto da criança e adolescente (ECA).</font></p>

| faixa | categoria |
| --- | --- |
| 0-11 | criança |
| 12-17  | adolescente |
| 18-29 | jovem |
| 30-59 | adulto |
| acima de 60 | idoso |

In [ ]:
labels = ['criança', 'adolescente', 'jovem', 'adulto', 'idoso']
classes = [0, 11, 17, 29, 59, 99]
df['faixa_etaria'] = pd.cut(x=df['idade'], bins=classes, labels=labels)

### Categorização variável data

In [ ]:
df['data'] = pd.to_datetime(df['data'])
df['estacao_do_ano'] = df['data'].apply(mapEstacaoAno)

### Seleção de features

In [ ]:
colunas = [
    'fase_dia',
    'tipo_acidente_agrupado',
    'causa_acidente',
    'condicao_metereologica_agrupado',
    'tracado_via',
    'sexo',
    'faixa_etaria',
    'estado_fisico',
    'estacao_do_ano',
    'dia_semana_agrupado',
    'tipo_pista',
    'porte_veiculo'
]

dados = df[colunas]
dados.head(1)

### Mineração de agrupamento